In [11]:
import json
import pandas as pd

## Import dataset 

In [12]:
data = pd.read_json('D:/Documents/IIT/3rd year/deep learning/cw p2/yelp dataset/yelp_academic_dataset_review.json', orient="records", lines= True, nrows=3000)


## Exploring Data

In [13]:
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [14]:
data.describe()

,stars,useful,funny,cool
count,3000.000000,3000.000000,3000.000000,3000.000000
mean,3.824667,0.871667,0.260000,0.333000
std,1.365728,1.639744,0.938808,0.934025
min,1.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,0.000000,0.000000
75%,5.000000,1.000000,0.000000,0.000000
max,5.000000,23.000000,19.000000,12.000000


In [15]:
data.isnull().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [16]:
data.nunique()


review_id      3000
user_id        2941
business_id    1878
stars             5
useful           18
funny            13
cool             12
text           3000
date           3000
dtype: int64

In [17]:
data.duplicated().sum()

0

In [18]:
data.shape

(3000, 9)